In [19]:
import numpy
import pandas as pd
import re

## Data Loading and exploration

### NOTE :- Please provide the path of restaurant_hours.csv file instead of 'C:\Users\Tarun\Downloads\abhishek\helpshift\restaurant_hours.csv'

In [31]:
data=pd.read_csv(r'C:\Users\Tarun\Downloads\abhishek\helpshift\restaurant_hours.csv',names=['Restaurant_Name','Timing'])
data.head(5)

,Restaurant_Name,Timing
0,Kushi Tsuru,Mon-Sun 11:30 am - 9 pm
1,Osakaya Restaurant,"Mon-Thu, Sun 11:30 am - 9 pm / Fri-Sat 11:30 ..."
2,The Stinking Rose,"Mon-Thu, Sun 11:30 am - 10 pm / Fri-Sat 11:30..."
3,McCormick & Kuleto's,"Mon-Thu, Sun 11:30 am - 10 pm / Fri-Sat 11:30..."
4,Mifune Restaurant,Mon-Sun 11 am - 10 pm


#Data Clean up

###Column name as Follows :- 
###index :- Day | 0 :- Restaurant_Name | btime :- Begin Time | etime :- End Time

In [21]:
df=pd.concat([pd.Series(row['Restaurant_Name'], row['Timing'].split('/'))              
                    for _, row in data.iterrows()]).reset_index()
df['etime']=df['index']

for i in range(len(df['etime'])):
    temp = df['etime'][i]
    pos = temp.rfind('-')
    df['etime'][i] = temp[pos+1:]
    df['index'][i] = temp[:pos-1]
    


df['btime']=df['index']
for i in range(len(df['btime'])):
    temp = df['btime'][i]
    pos = temp.rfind('-')
    
    if pos>0 :
        pos1 = temp.rfind(',')
        if pos1-pos>0:
            df['btime'][i] = temp[pos1+5:]
            df['index'][i] = temp[:pos1]
        else:
            df['btime'][i] = temp[pos+4:]
            df['index'][i] = temp[:pos+4]
    else:
        df['btime'][i] = temp[4:]
        df['index'][i] = temp[:4]
        
df.head(2)

,index,0,etime,btime
0,Mon-Sun,Kushi Tsuru,9 pm,11:30 am
1,Mon-Thu,Osakaya Restaurant,9 pm,11:30 am


In [22]:
for i in range(len(df['index'])):
    temp = df['index'][i]
    if temp == 'Mon-Sun' :
        df['index'][i] = temp.replace('Mon-Sun','Mon,Tue,Wed,Thu,Fri,Sat,Sun')
    elif temp == 'Mon-Sat' :
        df['index'][i] = temp.replace('Mon-Sat','Mon,Tue,Wed,Thu,Fri,Sat')
    elif temp == 'Mon-Fri' :
        df['index'][i] = temp.replace('Mon-Fri','Mon,Tue,Wed,Thu,Fri')
    elif temp == 'Mon-Thu' :
        df['index'][i] = temp.replace('Mon-Thu','Mon,Tue,Wed,Thu')
    elif temp == 'Mon-Wed' :
        df['index'][i] = temp.replace('Mon-Wed','Mon,Tue,Wed')
    elif temp == 'Wed-Sun' :
        df['index'][i] = temp.replace('Wed-Sun','Wed,Thu,Fri,Sat,Sun')
    elif temp == 'Fri-Sun' :
        df['index'][i] = temp.replace('Fri-Sun','Fri,Sat,Sun')
    else:
        df['index'][i] = temp.replace('-',',')

### Converting begin time and end time in 24 hr Format 

In [23]:
def convert_time(str):
    for i in range(len(str)):
        etime = str[i]
        etime = etime.replace(' ','')
        if etime.replace(' ','')[-2:] == 'pm':
            if etime[:len(etime)-2] == '12' :
               str[i] = 12
            elif etime[:len(etime)-2] == '12:30' :
                str[i] = 12.30
            else:
                str[i] = float(etime.replace(':','.')[:len(etime)-2])+12
        else:
            if etime[:len(etime)-2] == '12' :
               str[i] = 24
            elif etime[:len(etime)-2] == '12:30' :
                str[i] = 0.30
            else:
                str[i] = float(etime.replace(':','.')[:len(etime)-2])
            
convert_time(df['etime'])
convert_time(df['btime'])
df.head(5)

,index,0,etime,btime
0,"Mon,Tue,Wed,Thu,Fri,Sat,Sun",Kushi Tsuru,21,11.3
1,"Mon,Tue,Wed,Thu",Osakaya Restaurant,21,11.3
2,"Fri,Sat",Osakaya Restaurant,21.3,11.3
3,"Mon,Tue,Wed,Thu",The Stinking Rose,22,11.3
4,"Fri,Sat",The Stinking Rose,23,11.3


### Calculating total_open_time

In [24]:
df['total_open_time'] = df['btime']
def total_time(btime,etime):
    for i in range(len(btime)):
        if etime[i]>btime[i]:
            temp = str(btime[i])
            
            if temp.rfind('.')>0 : 
                temp = str(etime[i]- btime[i])
                temp = float(temp.replace('.7','.3'))
                df['total_open_time'][i] = temp
            else:
                df['total_open_time'][i] = etime[i]-btime[i]
        else:
            df['total_open_time'][i] = 24 - btime[i] + etime[i]
            
total_time(df['btime'],df['etime'])
df.head(5)

,index,0,etime,btime,total_open_time
0,"Mon,Tue,Wed,Thu,Fri,Sat,Sun",Kushi Tsuru,21,11.3,9.3
1,"Mon,Tue,Wed,Thu",Osakaya Restaurant,21,11.3,9.3
2,"Fri,Sat",Osakaya Restaurant,21.3,11.3,10
3,"Mon,Tue,Wed,Thu",The Stinking Rose,22,11.3,10.3
4,"Fri,Sat",The Stinking Rose,23,11.3,11.3


In [25]:
gb=df.groupby('etime')
gc=gb.count()
gc

,index,0,btime,total_open_time
etime,,,,
0.3,3,3,3,3
1.0,1,1,1,1
1.3,2,2,2,2
2.0,1,1,1,1
4.0,1,1,1,1
15.0,1,1,1,1
21.0,5,5,5,5
21.3,9,9,9,9
22.0,31,31,31,31


In [26]:
gb=df.groupby('btime')
gc=gb.count()
gc

,index,0,etime,total_open_time
btime,,,,
7.0,1,1,1,1
8.0,1,1,1,1
9.0,4,4,4,4
9.3,1,1,1,1
10.0,7,7,7,7
10.3,1,1,1,1
11.0,31,31,31,31
11.3,26,26,26,26
12.0,3,3,3,3


In [27]:
gb=df.groupby('total_open_time')
gc=gb.count()
gc

,index,0,etime,btime
total_open_time,,,,
4.0,1,1,1,1
4.3,2,2,2,2
5.0,3,3,3,3
6.0,1,1,1,1
7.3,1,1,1,1
8.0,1,1,1,1
8.3,2,2,2,2
8.7,1,1,1,1
9.3,3,3,3,3


## Creating function get_open_restaurants whose input parameter is DateTime and output is List of Open Restaurant that DateTime

### Input DateTime Format must be in 'YYYY-MM-DD HH12:MI:SS AM/PM'
### Ex :- '2016-08-18 01:35:00 AM'

In [10]:
def get_open_restaurants(var):
    rest = []
    day_no=pd.Timestamp(var).weekday()
    hour=pd.Timestamp(var).hour
    minute=pd.Timestamp(var).minute
    t = str(hour) + ':'+ str(minute)
    t = float(t.replace(':','.'))
    
    if day_no == 0 :
        rval = r"\b(?=\w)" + re.escape('Mon') + r"\b(?!\w)"
    elif day_no == 1 :
        rval = r"\b(?=\w)" + re.escape('Tue') + r"\b(?!\w)"
    elif day_no == 2 :
        rval = r"\b(?=\w)" + re.escape('Wed') + r"\b(?!\w)"
    elif day_no == 3 :
        rval = r"\b(?=\w)" + re.escape('Thu') + r"\b(?!\w)"
    elif day_no == 4 :
        rval = r"\b(?=\w)" + re.escape('Fri') + r"\b(?!\w)"
    elif day_no == 5 :
        rval = r"\b(?=\w)" + re.escape('Sat') + r"\b(?!\w)"
    else :
        rval = r"\b(?=\w)" + re.escape('Sun') + r"\b(?!\w)"
        
    for i in range(len(df['etime'])):
        if t>4:
            if (df['btime'][i]<=t ) and (df['etime'][i]>=t or df['etime'][i]<=4) and re.findall(rval,df['index'][i]):
                rest.append(df[0][i])
        else:
            if (df['etime'][i]>=t and df['etime'][i]<=4) and re.findall(rval,df['index'][i]):
                rest.append(df[0][i])
                
    return rest
        

In [11]:
test=get_open_restaurants('2016-08-18 12:35:00 AM')
print test

['Sudachi', "Naan 'N' Curry", 'Thai Stick Restaurant', 'Marrakech Moroccan Restaurant']


In [12]:
test1=get_open_restaurants('2016-08-15 12:35:00 PM')
print test1

['Kushi Tsuru', 'Osakaya Restaurant', 'The Stinking Rose', "McCormick & Kuleto's", 'Mifune Restaurant', 'The Cheesecake Factory', 'New Delhi Indian Restaurant', 'Iroha Restaurant', 'Rose Pistola', "Alioto's Restaurant", 'Canton Seafood & Dim Sum Restaurant', 'All Season Restaurant', 'Bombay Indian Restaurant', "Sam's Grill & Seafood Restaurant", '2G Japanese Brasserie', 'Restaurant Lulu', 'Hanuri', 'Herbivore', 'Penang Garden', "John's Grill", 'Quan Bac', 'Bamboo Restaurant', 'Burger Bar', 'Blu Restaurant', "Naan 'N' Curry", 'Shanghai China Restaurant', 'Tres', 'Isobune Sushi', 'Viva Pizza Restaurant', 'Far East Cafe', 'Parallel 37', 'Bai Thong Thai Cuisine', 'Alhamra', 'A-1 Cafe Restaurant', "Nick's Lighthouse", 'Paragon Restaurant & Bar', 'Chili Lemon Garlic', 'Bow Hon Restaurant', 'San Dong House', 'Thai Stick Restaurant', "Cesario's", 'Colombini Italian Cafe Bistro', 'Sabella & La Torre', 'Soluna Cafe and Lounge', 'Tong Palace', 'India Garden Restaurant', 'Sapporo-Ya Japanese Resta

In [13]:
test2=get_open_restaurants('2016-08-18 01:35:00 AM')
print test2

["Naan 'N' Curry", 'Marrakech Moroccan Restaurant']


In [14]:
test3=get_open_restaurants('2016-08-18 5:35:00 AM')
print test3

[]


In [15]:
test4=get_open_restaurants('2016-08-18 9:35:00 AM')
print test4

['Herbivore', 'Tong Palace', "Santorini's Mediterranean Cuisine"]


In [16]:
df.sort('total_open_time',ascending=1)

,index,0,etime,btime,total_open_time
24,Sat,Sam's Grill & Seafood Restaurant,21,17,4
62,Sat,Paragon Restaurant & Bar,22,17.3,4.3
12,Sun,New Delhi Indian Restaurant,22,17.3,4.3
75,Sat,Soluna Cafe and Lounge,22,17,5
71,"Sat,Sun",Colombini Italian Cafe Bistro,22,17,5
64,"Sat,Sun",Chili Lemon Garlic,22,17,5
85,Sun,Kyoto Sushi,22.3,16.3,6
29,"Mon,Tue,Wed",Sudachi,0.3,17,7.3
47,"Sat,Sun",Blu Restaurant,15,7,8
30,"Thu,Fri",Sudachi,1.3,17,8.3


## Finding longest open hours Restaurant (assuming total_open_time >= 12)

###Explore or Get List of Restaurant name which operats more than 12 hrs. 

In [17]:
df[df['total_open_time']>=12]

,index,0,etime,btime,total_open_time
8,"Mon,Tue,Wed,Thu",The Cheesecake Factory,23,11,12
9,"Fri,Sat",The Cheesecake Factory,0.3,11,13.3
10,Sun,The Cheesecake Factory,23,10,13
17,"Mon,Tue,Wed,Thu,Fri,Sat,Sun",Alioto's Restaurant,23,11,12
21,"Sat,Sun",All Season Restaurant,21.3,9.3,12
26,"Fri,Sat",2G Japanese Brasserie,23,11,12
33,"Mon,Tue,Wed,Thu,Fri,Sat,Sun",Hanuri,24,11,13
34,"Mon,Tue,Wed,Thu",Herbivore,22,9,13
35,"Fri,Sat",Herbivore,23,9,14
37,"Fri,Sat",Penang Garden,22.3,10,12.3


## Finding Shortest open hours Restaurant (assuming total_open_time <= 8)

###Explore or Get List of Restaurant name which operats less than 8 hrs.

In [18]:
df[df['total_open_time']<=8]

,index,0,etime,btime,total_open_time
12,Sun,New Delhi Indian Restaurant,22,17.3,4.3
24,Sat,Sam's Grill & Seafood Restaurant,21,17,4
29,"Mon,Tue,Wed",Sudachi,0.3,17,7.3
47,"Sat,Sun",Blu Restaurant,15,7,8
62,Sat,Paragon Restaurant & Bar,22,17.3,4.3
64,"Sat,Sun",Chili Lemon Garlic,22,17,5
71,"Sat,Sun",Colombini Italian Cafe Bistro,22,17,5
75,Sat,Soluna Cafe and Lounge,22,17,5
85,Sun,Kyoto Sushi,22.3,16.3,6


### Finding Time for Breakfast,Lunch and Dinner

In [28]:
gb=df.groupby('etime')
gc=gb.count()
gc

,index,0,btime,total_open_time
etime,,,,
0.3,3,3,3,3
1.0,1,1,1,1
1.3,2,2,2,2
2.0,1,1,1,1
4.0,1,1,1,1
15.0,1,1,1,1
21.0,5,5,5,5
21.3,9,9,9,9
22.0,31,31,31,31


### Mostly People do dinner prior to 10PM

In [30]:
gb=df.groupby('btime')
gc=gb.count()
gc

,index,0,etime,total_open_time
btime,,,,
7.0,1,1,1,1
8.0,1,1,1,1
9.0,4,4,4,4
9.3,1,1,1,1
10.0,7,7,7,7
10.3,1,1,1,1
11.0,31,31,31,31
11.3,26,26,26,26
12.0,3,3,3,3


### Breakffast Timing:- In between 7 to 11 and Lunch Timing:- In between 12 to 15